In [20]:
# 企業の財務情報を取得してデータフレームに格納する関数を定義
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

def make_table(com_id):
    date = []
    var_dic = {}
    url = 'https://www.buffett-code.com/company/' + com_id + '/financial'
    html=requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    tables=soup.find_all("table",class_ = "table custom-table table-hover table-condensed")

    for i in tables[0].find('thead').find_all('th')[1:]:
        date.append(i.get_text().replace('年','/').replace('月期',''))
    var_dic['date'] = date
    var_dic['id'] = com_id
    
    for table in tables:
        make_df(table,var_dic)
        
    return pd.DataFrame(var_dic).convert_objects(convert_numeric=True).replace('-',np.nan)

def make_df(table,var_dic):
    for j in table.find('tbody').find_all('tr'):
        value = [value.get_text().replace(',','').replace('%','').replace('人','').replace('倍','').replace('株','').replace('ヶ月','').replace('日','') for value in j.find_all('td')]
        var_dic[value[0]] = value[1:]
    return var_dic

In [62]:
# idsに定義した証券コードの財務データをsqliteに格納をする
ids = ['8411','7203','1301','1332','1333','1352','1376','1377']
conn = sqlite3.connect('db.sqlite3')
for com_id in ids:
    # 作成したデータをtempとして格納
    make_table(com_id).to_sql('zaimu_data_temp',conn,if_exists='replace',index=None)
    conn.commit()
    try :
        pd.read_sql_query('SELECT * FROM zaimu_data', conn)
    except :
        make_table(com_id).to_sql('zaimu_data',conn,if_exists='replace',index=None)
        conn.commit()
    add_data = pd.read_sql_query('SELECT a.* FROM zaimu_data_temp AS a LEFT OUTER JOIN zaimu_data AS b ON a.date = b.date AND a.id = b.id WHERE b.date IS NULL', conn)
    add_data.to_sql('zaimu_data',conn,if_exists='append',index=None)
    conn.commit()
conn.close()

C:\Users\hirou\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\hirou\Anaconda3\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [63]:
# 格納したデータを確認する
conn = sqlite3.connect('db.sqlite3')
df=pd.read_sql_query('SELECT * FROM zaimu_data', conn)
print(df)
conn.close()

      date    id         売上高    前期比        売上原価      売上総利益 売上総利益率        販管費  \
0   2017/3  8411   3292900.0   90.0         NaN        NaN               NaN   
1   2018/3  8411   3561125.0   95.5         NaN        NaN               NaN   
2   2019/3  8411   3925649.0   16.7         NaN        NaN               NaN   
3   2020/3  8411         NaN  486.7         NaN        NaN   None        NaN   
4   2016/3  7203  28403118.0  106.5  22600000.0  5800000.0   20.4  2940000.0   
5   2017/3  7203  27597193.0   79.2  22700000.0  4860000.0   17.6  2870000.0   
6   2018/3  7203  29400000.0  136.1  23900000.0  5490000.0   18.7  3090000.0   
7   2019/3  7203  30000000.0   90.4         NaN        NaN   None        NaN   
8   2017/3  1301    236561.0  134.6    213292.0    23268.0    9.8    19544.0   
9   2018/3  1301    254783.0  132.6    231180.0    23603.0    9.3    19536.0   
10  2019/3  1301    256151.0   90.8    232446.0    23705.0    9.3    19873.0   
11  2020/3  1301    286000.0  130.4     

In [64]:
# 参考: zaimu_dataから各証券番号ごとの売上最大値を求める
conn = sqlite3.connect('db.sqlite3')
df=pd.read_sql_query('SELECT id,date,MAX(売上高) AS 最大値 FROM zaimu_data GROUP BY id', conn)
print(df)
conn.close()

     id    date         最大値
0  1301  2020/3    286000.0
1  1332  2019/3    712111.0
2  1333  2020/3    940000.0
3  1352  2020/3     79500.0
4  1376  2019/5     60000.0
5  1377  2019/5     63800.0
6  7203  2019/3  30000000.0
7  8411  2019/3   3925649.0
